In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# Definir vectores iniciales
Er_1 = np.array([[1], [0]])  
Er_2 = np.array([[1], [1]])  
Es = np.array([[1], [2 * np.exp(1j * np.pi/4)]])

# Calcular intensidades
info_1, i_1 = pi.I(Er_1, Es)
info_2, i_2 = pi.I(Er_2, Es)

# Cálculo de componentes de Es
Esx = info_1[1] / np.abs(Er_1[0])
Esy = np.sqrt(info_1[0] - Esx**2 - Er_1[0]**2)
Es_c = np.array([[Esx], [Esy]])

# Cálculo del numerador y denominador
norm_Er2_sq = np.linalg.norm(Er_2)**2
norm_Es_c_sq = np.linalg.norm(Es_c)**2

numerador = (norm_Er2_sq + norm_Es_c_sq) * np.tan(info_2[2])[0]
denominador = norm_Er2_sq + norm_Es_c_sq + 2 * np.abs(Er_2[0])[0] * np.abs(Es_c[0])[0] * np.abs(Er_2[1])[0] * np.abs(Es_c[1])[0]

# Definir variable simbólica
phi_s = sp.Symbol('Delta_phi_s')

# Definir matrices simbólicas
Es_c_sp = sp.Matrix([Esx[0], Esy[0]])  
Er_2_sp = sp.Matrix([Er_2[0,0], Er_2[1,0]]) 

# Expresión para la fase
expr = sp.atan2(-Er_2_sp[1] * Es_c_sp[1] * sp.sin(-phi_s), 
                Er_2_sp[0] * Es_c_sp[0] + Er_2_sp[1] * Es_c_sp[1] * sp.cos(-phi_s)) - info_2[2]

# Resolver para Δφ_s
solucion = sp.solve(expr, phi_s)
delta_phi_s = solucion[0] if solucion else "No se encontraron soluciones simbólicas."

# Resultados
print(f"Esx: {Esx}")
print(f"Esy: {Esy}")
print(f"Delta_phi_s: {delta_phi_s}")
print(f"Ángulo de Es[1]: {np.angle(Es[1,0])}")

In [ ]:
#import sympy as sp

# Definir variables simbólicas
#vartheta = sp.Symbol('vartheta')  
#phi_s = sp.Symbol('Delta_phi_s')  

# Definir elementos simbólicos de las matrices
#Es_c_1, Es_c_2 = sp.symbols('Es_c_1 Es_c_2')
#Er_2_1, Er_2_2 = sp.symbols('Er_2_1 Er_2_2')

# Expresión general con variables simbólicas
#expr = sp.atan2(-Er_2_2 * Es_c_2 * sp.sin(-phi_s), 
#                Er_2_1 * Es_c_1 + Er_2_2 * Es_c_2 * sp.cos(-phi_s)) - vartheta

# Resolver para Δφ_s
#solution = sp.solve(expr, phi_s)

# Imprimir solución simbólica
#print(solution if solution else "No se encontraron soluciones simbólicas.")
